In [1]:
import urllib.request
import requests
import pandas as pd
import gspread
import re

In [2]:
def df_to_sheet(dataframe, sheet): # input dataframe and sheet string
    gc = gspread.oauth()
    sh = gc.open(sheet)
    worksheet = sh.get_worksheet(0) # get first worksheet
    worksheet.update('A2', [dataframe.columns.values.tolist()] + dataframe.values.tolist()) # add dataframe to sheet
    # could take 2 inputs, first location where to paste, and second what to paste

In [3]:
def keap_df(all_leads_txt): # input file name as string
    with open(all_leads_txt) as file: # remove extra spaces, separate each element based on newline, and add "endofline"
        lines = file.readlines()
        lines = [line.rstrip() + "endofline" for line in lines]
        
    # join each element, separated by a comma, then split them into different elements based on 'endofline'
    # then each element will be a single lead. now split each lead into different portions
    n_data = [l.split('\t') for l in ','.join(lines).split('endofline')]
    
    # remove blank lists
    n_data_correct = []
    for i in range(len(n_data)):
        if len(n_data[i]) > 2:
            n_data_correct.append(n_data[i])
    n_data = n_data_correct
    
    # remove ',' element that starts each list but first
    for i in range(len(n_data)):
        n_data[i].pop(0)
    
    # separate leads into those that are full and those missing something (like an email)
    n_data_full = []
    n_data_missing = []
    for i in range(len(n_data)):
        if len(n_data[i]) >= 7:
            n_data_full.append(n_data[i])
    else:
        n_data_missing.append(n_data[i])
        
    # create blank lists for each column
    opportunity = []
    stage = []
    date_created = []
    last_updated = []
    phone = []
    contact_name = []
    email = []
    notes = []
    
    # add leads to blank lists
    for lead in n_data_full:
        opportunity.append(lead[0])
        stage.append(lead[1])
        date_created.append(lead[2])
        last_updated.append(lead[3])
        phone.append(lead[4])
        contact_name.append(lead[5])
        email.append(lead[6])
        if len(lead) > 7:
            notes.append(lead[7])
        else:
            notes.append('n/a')
            
    # create dictionary
    data_dict = {'Opportunity':opportunity, 'Stage':stage, 'Date created': date_created, 'Last updated':last_updated, 'Phone':phone, 'Contact name':contact_name, 'Email':email, 'Notes':notes}
    
    # create dataframe
    df = pd.DataFrame(data_dict)
    
    return df, data_dict

In [4]:
page_one, page_one_dict = keap_df('1st_page.txt')
df_to_sheet(page_one, '1st_page')

In [5]:
reach, reach_dict = keap_df('reached_us.txt')

In [6]:
def reach_us_dict(dictionary): # input dictionary
    index_list = []
    date_list = []
    for i in range(len(dictionary['Notes'])):
        note = dictionary['Notes'][i]
        stages = dictionary['Stage'][i] 
        if (note[:10] == "reached_us") and (stages in ['SENT APP', 'Submission', 'Approved']):
            index_list.append(i)
            date_list.append(note[11:22])
    
    opportunity = []
    stage = []
    date_created = []
    phone = []
    contact_name = []
    email = []
    notes = []
    
    for i in index_list:
        opportunity.append(dictionary['Opportunity'][i])
        stage.append(dictionary['Stage'][i])
        date_created.append(dictionary['Date created'][i])
        phone.append(dictionary['Phone'][i])
        contact_name.append(dictionary['Contact name'][i])
        email.append(dictionary['Email'][i])
        notes.append(dictionary['Notes'][i])
    
    reached_us_dict = {'Opportunity':opportunity, 'Stage':stage, 'Date created': date_created, 'Reached us':date_list, 'Phone':phone, 'Contact name':contact_name, 'Email':email, 'Notes':notes}
    
    return reached_us_dict

In [7]:
to_do_dict = reach_us_dict(reach_dict)
to_do_df = pd.DataFrame(to_do_dict)
df_to_sheet(to_do_df, 'to_do')

In [29]:
match = re.search(r"'follow_up' (\d.+)", 'follow_up 12/27/2021 ')

In [30]:
match.group(0)

AttributeError: 'NoneType' object has no attribute 'group'